# ENGIE - GEMS 
## Evaluación de las habilidades de gestión de datos y digitalización. Conciliación datos

---
Prueba práctica del proceso de selección desarrollada por Alba Durán Merinero

## 1. Configuración de paths y librerías 

In [1]:
import funciones_caso_practico as f
import os

main_path = "datos/" 
facturas_path = main_path + "facturas_agiekey.xlsx"
mapping_path = main_path + "Mapping.xlsx"
deals_path = main_path + "Deals_Marzo_2024/"

## 2. Configuración de parámetros de la conciliación 

Es necesario establecer los criterios del análisis a llevar a cabo. Se necesita informar del Año, Mes, Origen y Servicio Facturado de las facturas a conciliar.
Toda la información debe aportarse en formato string (texto entrecomillado)

**IMPORTANTE:**
En este notebook se han configurado los parámetros para la resolución del apartado 2b.

In [2]:
#PARAMETROS PARA REPRODUCIR LOS RESULTADOS DEL APARTADO 2a
#Fecha
ano ='2024'
mes = '03'  #codificamos mes de marzo 
#Origen y Servicio Facturado a conciliar, expresado igual que en el fichero Facturas_agiekey
origen = 'TVB'
servicio_facturado = 'Almacenamiento TVB'

## 3. Fichero facturas 

In [3]:
#Carga del fichero de facturas
facturas = f.carga_xlsx(facturas_path)
print('Número total de facturas: ' + str(facturas.shape[0]))

Número total de facturas: 14


In [4]:
#Visualización df facturas inicial
facturas.head(2)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,Estado,FechaEstado,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones
0,2024264672,ESA86484292,"ENAGAS GTS, SAU",60,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,123195.13,EUR,Recibida,4/3/24,4/3/24,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN
1,2024264671,ESA86484292,"ENAGAS GTS, SAU",60,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,111963.62,EUR,Recibida,4/3/24,4/3/24,eMail,344991,AVB,Servicio agregado AVB,NaN,NaN


### 3.1 Filtrado de facturas

In [5]:
#Filtro de Facturas según el Origen y Servicio Facturado a conciliar
facturas = facturas[(facturas['Origen'] == origen) & (facturas['ServicioFacturado'] == servicio_facturado)]
print('Número de facturas correspondientes al Origen y Servicio Facturado: ' + str(facturas.shape[0]))

Número de facturas correspondientes al Origen y Servicio Facturado: 11


In [6]:
#Visualización df facturas filtrando el concepto de Origen y Servicio Facturado correspondiente
facturas.head(2)

,NumeroFactura,Emisor,RazonSocialEmisor,DepEmisor,Receptor,RazonSocialReceptor,DepReceptor,FechaFactura,Importe,Moneda,Estado,FechaEstado,FechaRegistro,Destino,Contrato,Origen,ServicioFacturado,¿Verificada por BO?,Observaciones
3,2024164592,ESA86484292,"ENAGAS GTS, SAU",50,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,1375.12,EUR,Recibida,4/3/24,4/3/24,eMail,344991,TVB,Almacenamiento TVB,NaN,NaN
4,2024164591,ESA86484292,"ENAGAS GTS, SAU",50,ESB82508441,ENGIE ESPAÑA SLU,NaN,4/3/24,1787.43,EUR,Recibida,4/3/24,4/3/24,eMail,344991,TVB,Almacenamiento TVB,NaN,NaN


### 3.2 Resumen de facturas

In [7]:
#Resumen de facturas
resumen_fact = f.global_info(facturas)

#Prints de la información
print('Número de facturas correspondientes al Origen y Servicio Facturado: ' + str(resumen_fact['NumeroFacturasValidas'][0]))
print('Número de clientes: ' + str(resumen_fact['NumeroClientes'][0]))
print('Facturación global del concepto: ' + str(resumen_fact['FacturacionTotal'][0]))

Número de facturas correspondientes al Origen y Servicio Facturado: 11
Número de clientes: 1
Facturación global del concepto: 72435.76EUR


## 4. Carga del fichero de mapping 

In [8]:
#Carga fichero mapping
mapping = f.carga_xlsx(mapping_path)

In [9]:
#Filtro de dataset Mapping según el Origen y Servicio Facturado a conciliar en Facturas
mapping = mapping[(mapping['Origen'] == origen) & (mapping['Servicio facturado'] == servicio_facturado)]
#Creación nueva columna
mapping = f.combcols(mapping, 'FileName', ['Portfolio','Commodity','DealType'])
#En caso de filas 100% duplicadas se mantiene solo la primera
mapping = mapping.drop_duplicates(keep='first')
#Visualización del df mapping filtrando el concepto de Origen y Servicio Facturado correspondiente
mapping

,Origen,Servicio facturado,Portfolio,Commodity,DealType,FileName
16,TVB,Almacenamiento TVB,JVLNG,CTV,STOK,JVLNG_CTV_STOK


### 4.1 Selección a través del mapping del archivo de Deals corresponiente para conciliar las facturas

In [10]:
#Lista de archivos que coinciden según el origen
file_names = list(mapping['FileName'])
#Obtencion de lista de archivos en el directorio de los deals
list_files = os.listdir(deals_path)
#Filtro de archivos que correspondientes
filtered_files = [file for file in list_files if (file.startswith(file_names[0])) & (file.endswith('.xlsx'))]
#Construcción del path
fil_file_path = deals_path + filtered_files[0]


In [11]:
#fichero o ficheros leidos para el análisis de Deals
print(filtered_files)

['JVLNG_CTV_STOK_MARZO_2024_05_23_14_12.xlsx']


## 5. Carga fichero de deals

In [12]:
deals = f.carga_xlsx(fil_file_path)
print('Numero total de deals: '+ str(deals.shape[0]))
#Visualización df deals inicial
deals.head(2)

Numero total de deals: 42


,Unnamed: 0,AggregatedKey,ApplicationPeriod,BookingSource,BOValidated,BOValidationDate,BOValidator,CancelsDealId,Capacity,CapacityFrequency,...,ExerciseDate,ExerciseType,Nature,OptionNature,OptionType,PeriodInfoCollection,Script,TemplateId,TemplateScript,OrderId
0,0,"42563586,2024/05/20 13:35:59,GEFWD",010524,MET,True,2024-05-20T13:35:59+02:00,GGH410,0,90000,1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"42518127,2024/05/20 13:34:13,GEFWD",040624,MET,True,2024-05-20T13:34:13+02:00,GGH410,0,50000,1440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 5.1 Selección de Deals con Status Alive y DealType correspondiente

In [13]:
#Extración de deals con Status == 'Alive', DealType == 'STOK'
deals = deals[(deals['Status']=='Alive') & (deals['DealType']=='STOK')]
print('Numero total de deals con Status Alive y DealType STOK: '+ str(deals.shape[0]))


Numero total de deals con Status Alive y DealType STOK: 4


### 5.2 Análisis de los sub-flows
La información de cada uno de los flows que se engloban en un Deal, se puede obtener del campo ['Flow'], el cual se encuentra en formato json

In [15]:
#Extracción de información de sub-flows de los registros correspondientes
flow_df = f.extract_df_from_json_src(df= deals, col= 'Flows',deal_col= 'Id')
print('Número total de sub-flows: ' + str(flow_df.shape[0]))

Generando dataframe para deal:  39980344
Generando dataframe para deal:  39705811
Generando dataframe para deal:  37761994
Generando dataframe para deal:  31915597
Concatenando dataframes generados...
Número total de sub-flows: 120


In [16]:
#Selección de los flows del año y mes correspondientes a las facturas
flow_df = flow_df[flow_df['Date'].apply(lambda x: x.startswith(ano + '-' + mes))]
print('Número de sub-flows para análisis: ' + str(flow_df.shape[0]))
#Visualización df flows filtrando las fechas correspondientes al análisis
flow_df

Número de sub-flows para análisis: 4


,Deal_Id,SubFlow_Id,Date,Subcantidad
0,39980344,101112346,2024-03-20T00:00:00+01:00,0.00
1,39980344,101112345,2024-03-20T00:00:00+01:00,72435.51
78,37761994,94022896,2024-03-20T00:00:00+01:00,0.00
79,37761994,94022897,2024-03-20T00:00:00+01:00,0.00


In [17]:
agg_flow = flow_df.groupby('Deal_Id').agg({'Subcantidad':'sum'}).reset_index()
agg_flow = {col:value for col, value in zip(agg_flow['Deal_Id'], agg_flow['Subcantidad'])}

## 6. Calculo de totales 
Calculos y operaciones necesarios para la obtención de información requerida por BackOffice

### 6.1 Facturas de AGIKEY

In [18]:
facturas[['NumeroFactura','Importe', 'FechaFactura']]

,NumeroFactura,Importe,FechaFactura
3,2024164592,1375.12,4/3/24
4,2024164591,1787.43,4/3/24
5,2024164590,1930.23,4/3/24
6,2024164589,1472.13,4/3/24
7,2024164588,1965.66,4/3/24
8,2024164587,1041.67,4/3/24
9,2024164586,1030.97,4/3/24
10,2024164585,1017.59,4/3/24
11,2024164584,1015.95,4/3/24
12,2024164583,718.6,4/3/24


In [19]:
#Cantidades total de las facturas
TotalQuantityAGIKEY = facturas['Importe'].astype('float').sum()

### 6.2 Deals TRS

In [20]:
TotalQuantityTRS = flow_df['Subcantidad'].sum()

In [22]:
results_deals = deals[['Id', 'StartDeliveryDate', 'EndDeliveryDate']]
results_deals['Cantidad'] = results_deals['Id'].apply(lambda x: agg_flow[x] if x in agg_flow.keys()  else 0.0)
results_deals['Procedencia TRS'] = file_names[0]
results_deals = results_deals[['Id', 'Cantidad', 'StartDeliveryDate', 'EndDeliveryDate', 'Procedencia TRS']]
results_deals = results_deals.rename(columns={'Id':'Deal'})
results_deals

,Deal,Cantidad,StartDeliveryDate,EndDeliveryDate,Procedencia TRS
25,39980344,72435.51,2023-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,JVLNG_CTV_STOK
27,39705811,0.00,2024-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,JVLNG_CTV_STOK
30,37761994,0.00,2022-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,JVLNG_CTV_STOK
41,31915597,0.00,2022-03-30T00:00:00+02:00,2023-01-01T00:00:00+01:00,JVLNG_CTV_STOK


### 6.2 Matcheo

In [29]:
Importe_conciliado = TotalQuantityTRS
Importe_no_conciliado = TotalQuantityAGIKEY - Importe_conciliado


## 7. Output final

In [30]:
print('('+ origen + ', ' + servicio_facturado +')')
print('---> FACTURAS DE AGIKEY <---')
facturas[['NumeroFactura','Importe', 'FechaFactura']].reset_index(drop=True)

(TVB, Almacenamiento TVB)
---> FACTURAS DE AGIKEY <---


,NumeroFactura,Importe,FechaFactura
0,2024164592,1375.12,4/3/24
1,2024164591,1787.43,4/3/24
2,2024164590,1930.23,4/3/24
3,2024164589,1472.13,4/3/24
4,2024164588,1965.66,4/3/24
5,2024164587,1041.67,4/3/24
6,2024164586,1030.97,4/3/24
7,2024164585,1017.59,4/3/24
8,2024164584,1015.95,4/3/24
9,2024164583,718.6,4/3/24


In [31]:
print('---> DEALS TRS <---')
results_deals

---> DEALS TRS <---


,Deal,Cantidad,StartDeliveryDate,EndDeliveryDate,Procedencia TRS
25,39980344,72435.51,2023-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,JVLNG_CTV_STOK
27,39705811,0.00,2024-10-01T00:00:00+02:00,2026-09-30T00:00:00+02:00,JVLNG_CTV_STOK
30,37761994,0.00,2022-12-31T00:00:00+01:00,2024-12-31T00:00:00+01:00,JVLNG_CTV_STOK
41,31915597,0.00,2022-03-30T00:00:00+02:00,2023-01-01T00:00:00+01:00,JVLNG_CTV_STOK


In [32]:
deal_list = list(results_deals['Deal'].unique())
for i in deal_list:
    aux_df = flow_df[flow_df['Deal_Id'] == i]
    aux_df = aux_df.reset_index(drop=True)
    if aux_df.empty:
        print('Deal '+ str(i) + ': sin flows asociados para el mes de estudio')
    else:
        print('Deal ' + str(i) + ': ')    
        print(aux_df[['SubFlow_Id','Date','Subcantidad']])

Deal 39980344: 
   SubFlow_Id                       Date  Subcantidad
0   101112346  2024-03-20T00:00:00+01:00         0.00
1   101112345  2024-03-20T00:00:00+01:00     72435.51
Deal 39705811: sin flows asociados para el mes de estudio
Deal 37761994: 
   SubFlow_Id                       Date  Subcantidad
0    94022896  2024-03-20T00:00:00+01:00          0.0
1    94022897  2024-03-20T00:00:00+01:00          0.0
Deal 31915597: sin flows asociados para el mes de estudio


In [33]:
print(' -----------MATCHEO-------------')
print('TotalQuantityAGIKEY: ' + str(TotalQuantityAGIKEY) + ' --> Facturación Total')
print('TotalQuantityTRS: '+ str(TotalQuantityTRS) + ' --> Importe Conciliado')
print('TotalQuantityAGIKEY - TotalQuantityTRS: ' + str(TotalQuantityAGIKEY-TotalQuantityTRS) + ' --> Importe No Conciliado')

 -----------MATCHEO-------------
TotalQuantityAGIKEY: 72435.76000000001 --> Facturación Total
TotalQuantityTRS: 72435.51 --> Importe Conciliado
TotalQuantityAGIKEY - TotalQuantityTRS: 0.2500000000145519 --> Importe No Conciliado
